In [21]:
import pandas as pd
import numpy as np


# Load dataset
df = pd.read_csv("Downloads/OCTOBER 2025 ALL DATA/job_change_data.csv")
df.shape, df.columns.tolist()


((2129, 13),
 ['enrollee_id',
  'city',
  'city_development_index',
  'gender',
  'relevent_experience',
  'enrolled_university',
  'education_level',
  'major_discipline',
  'experience',
  'company_size',
  'company_type',
  'last_new_job',
  'training_hours'])

In [23]:
# Quick look at the data and types
display(df.head())
df.info()
df.describe(include='all').T

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,Oct-49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             2129 non-null   int64  
 1   city                    2129 non-null   object 
 2   city_development_index  2129 non-null   float64
 3   gender                  1621 non-null   object 
 4   relevent_experience     2129 non-null   object 
 5   enrolled_university     2098 non-null   object 
 6   education_level         2077 non-null   object 
 7   major_discipline        1817 non-null   object 
 8   experience              2124 non-null   object 
 9   company_size            1507 non-null   object 
 10  company_type            1495 non-null   object 
 11  last_new_job            2089 non-null   object 
 12  training_hours          2129 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 216.4+ KB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
enrollee_id,2129.0,NaN,NaN,NaN,16861.614843,9576.846029,3.0,8562.0,16816.0,25129.0,33353.0
city,2129,108,city_103,473,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city_development_index,2129.0,NaN,NaN,NaN,0.824984,0.125074,0.448,0.698,0.903,0.92,0.949
gender,1621,3,Male,1460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
relevent_experience,2129,2,Has relevent experience,1524,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enrolled_university,2098,3,no_enrollment,1519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education_level,2077,5,Graduate,1269,NaN,NaN,NaN,NaN,NaN,NaN,NaN
major_discipline,1817,6,STEM,1621,NaN,NaN,NaN,NaN,NaN,NaN,NaN
experience,2124,22,>20,383,NaN,NaN,NaN,NaN,NaN,NaN,NaN
company_size,1507,8,50-99,338,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
def convert_experience(val):
    if pd.isna(val): return np.nan
    if isinstance(val, (int, float)): return val
    s = str(val).strip()
    if s == '>20': return 21
    if s == '<1': return 0
    try:
        return int(s)
    except:
        return np.nan

def convert_company_size(val):
    # Convert "10-50", "10000+", "<10", "Oct-49" -> safely to numeric (use mean for ranges, strip non-digits otherwise)
    if pd.isna(val): return np.nan
    s = str(val).strip()
    # handle plus like "10000+"
    if '+' in s:
        try:
            return int(''.join(ch for ch in s if ch.isdigit()))
        except:
            return np.nan
    # handle ranges like "10-50"
    if '-' in s:
        parts = s.split('-')
        # try to cast both ends to int, otherwise NaN
        nums = []
        for p in parts:
            pnum = ''.join(ch for ch in p if ch.isdigit())
            if pnum:
                nums.append(int(pnum))
        if len(nums) == 2:
            return sum(nums)/2.0
        else:
            return np.nan
    # single number (or strings like "<10" or "10")
    digits = ''.join(ch for ch in s if ch.isdigit())
    if digits:
        return int(digits)
    return np.nan

def convert_last_new_job(val):
    if pd.isna(val): return np.nan
    s = str(val).strip()
    if s == 'never': return 0
    if s == '>4': return 5
    try:
        return int(s)
    except:
        return np.nan

# Apply conversions
df['experience'] = df['experience'].apply(convert_experience).astype(float)
df['company_size'] = df['company_size'].apply(convert_company_size).astype(float)
df['last_new_job'] = df['last_new_job'].apply(convert_last_new_job).astype(float)

# Verify conversions
df[['experience', 'company_size', 'last_new_job']].head(10)


,experience,company_size,last_new_job
0,9.0,10.0,1.0
1,5.0,NaN,1.0
2,0.0,NaN,0.0
3,11.0,NaN,1.0
4,21.0,10000.0,5.0
5,10.0,NaN,2.0
6,0.0,300.0,1.0
7,21.0,NaN,5.0
8,14.0,NaN,4.0
9,3.0,74.5,1.0


In [31]:
## Question (1) Filter rows where City_Development_Index > 0.8 and Company_Size > 3


filtered_df = df[(df['city_development_index'] > 0.8) & (df['company_size'] > 3)]
print("Filtered rows shape:", filtered_df.shape)
filtered_df.head()


Filtered rows shape: (937, 13)


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9.0,10.0,NaN,1.0,21
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,21.0,10000.0,Pvt Ltd,5.0,72
13,24372,city_98,0.949,NaN,Has relevent experience,no_enrollment,Masters,STEM,4.0,300.0,Pvt Ltd,1.0,134
17,7463,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,Business Degree,2.0,74.5,Pvt Ltd,1.0,31
21,16001,city_103,0.920,NaN,Has relevent experience,no_enrollment,Graduate,STEM,7.0,10000.0,NaN,1.0,44


In [33]:
## Question (2) Use .iloc to select the first 10 rows and specific columns (Experience, Education_Level)

subset_iloc = filtered_df.iloc[:10][['experience', 'education_level']]
subset_iloc

,experience,education_level
0,9.0,Graduate
4,21.0,Graduate
13,4.0,Masters
17,2.0,Masters
21,7.0,Graduate
26,21.0,Masters
34,21.0,Masters
35,17.0,Masters
36,21.0,Graduate
37,21.0,Graduate


In [37]:
## Question 3 - Group by relevent_experience and calculate average city_development_index

avg_cdi_by_relevant = df.groupby('relevent_experience', dropna=False)['city_development_index'].mean().sort_values(ascending=False)
avg_cdi_by_relevant


relevent_experience
Has relevent experience    0.831000
No relevent experience     0.809828
Name: city_development_index, dtype: float64

In [39]:
## Question 4 - Group by company_size and count number of unique entries in last_new_job

unique_last_job_by_size = df.groupby('company_size')['last_new_job'].nunique(dropna=True).sort_index()
unique_last_job_by_size


company_size
10.0       6
74.5       6
300.0      6
749.5      6
2999.5     6
7499.5     6
10000.0    6
Name: last_new_job, dtype: int64

In [41]:
## Question 5 - Frequency distribution of company_type using value_counts()

company_type_freq = df['company_type'].value_counts(dropna=False)  # include NaN if you want
company_type_freq


company_type
Pvt Ltd                1141
NaN                     634
Public Sector           127
Funded Startup           97
Early Stage Startup      65
NGO                      53
Other                    12
Name: count, dtype: int64

In [43]:
## Question 6 - Find numerical columns with missing values and fill them with the column mean

# Identify numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Show missing counts
missing_numeric = df[numeric_cols].isnull().sum()
print("Numeric columns and missing counts:\n", missing_numeric[missing_numeric > 0])

# Fill numeric missing with mean (store a copy so original remains if you need it)
df_filled = df.copy()
for col in numeric_cols:
    if df_filled[col].isnull().any():
        mean_val = df_filled[col].mean()
        df_filled[col].fillna(mean_val, inplace=True)
        print(f"Filled {col} missing values with mean = {mean_val:.4f}")


Numeric columns and missing counts:
 experience        5
company_size    794
last_new_job     40
dtype: int64
Filled experience missing values with mean = 10.1050
Filled company_size missing values with mean = 2469.7090
Filled last_new_job missing values with mean = 2.0144


/var/folders/1d/m4bjh0fs4q31dy8sp0jbp0280000gp/T/ipykernel_2783/2673924373.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled[col].fillna(mean_val, inplace=True)


In [45]:
## Question 7 - Drop rows where >50% of columns are missing and document impact on dataset size

df_before_drop = df.copy()
initial_rows = df_before_drop.shape[0]

# threshold: keep rows with at least half non-NA values
thresh = int(np.floor(df_before_drop.shape[1] * 0.5)) + 1  # require >50% non-null; alternative: thresh = df.shape[1]*0.5
df_dropped = df_before_drop.dropna(thresh=thresh)

after_rows = df_dropped.shape[0]
dropped_rows = initial_rows - after_rows
pct_dropped = 100.0 * dropped_rows / initial_rows

print(f"Initial rows: {initial_rows}")
print(f"Rows after dropping rows with >50% missing: {after_rows}")
print(f"Rows dropped: {dropped_rows} ({pct_dropped:.2f}%)")


Initial rows: 2129
Rows after dropping rows with >50% missing: 2127
Rows dropped: 2 (0.09%)


In [50]:
## Question 8 - Query rows where Experience > 10 and Company_Size == 7


# Because company_size is float, equality may be exact if it converted to 7.0
query_result = df_filled[(df_filled['experience'] > 10) & (df_filled['company_size'] == 7)]
print("Query result shape:", query_result.shape)
display(query_result.head(20))


Query result shape: (0, 13)


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours


In [52]:
## Question 9 - Create new feature Experience_Gap = Experience - Last_New_Job


# Use the filled numeric DF (df_filled) so we avoid NaNs breaking math
df_filled['experience_gap'] = df_filled['experience'] - df_filled['last_new_job']
df_filled[['experience', 'last_new_job', 'experience_gap']].head(10)


,experience,last_new_job,experience_gap
0,9.0,1.0,8.0
1,5.0,1.0,4.0
2,0.0,0.0,0.0
3,11.0,1.0,10.0
4,21.0,5.0,16.0
5,10.0,2.0,8.0
6,0.0,1.0,-1.0
7,21.0,5.0,16.0
8,14.0,4.0,10.0
9,3.0,1.0,2.0


In [54]:
## Question 10 - Normalize city_development_index to 0–1 scale and explain benefit


# Min-max normalization to 0-1
min_cdi = df_filled['city_development_index'].min()
max_cdi = df_filled['city_development_index'].max()
df_filled['cdi_normalized'] = (df_filled['city_development_index'] - min_cdi) / (max_cdi - min_cdi)

# Quick check
df_filled[['city_development_index', 'cdi_normalized']].describe()

# Explanation (print)
print("Normalization (min-max) rescales CDI to [0,1].")
print("- Benefits: puts CDI on the same scale as other normalized features,")
print("  improves stability for algorithms sensitive to scale (distance-based methods, gradient-based models),")
print("  and makes coefficients easier to interpret when combining features.")


Normalization (min-max) rescales CDI to [0,1].
- Benefits: puts CDI on the same scale as other normalized features,
  improves stability for algorithms sensitive to scale (distance-based methods, gradient-based models),
  and makes coefficients easier to interpret when combining features.


In [56]:
## Question 11 - Create cdi_per and merge to original dataframe using pd.merge, then analyze combined insights


# Create cdi_per as percentage (0-100) from normalized cdi (or from raw cdi if you prefer)
df_filled['cdi_per'] = df_filled['cdi_normalized'] * 100

# Merge back to original df (by enrollee_id). We'll merge df (original) with df_filled[['enrollee_id','cdi_per']]
merged_df = pd.merge(df, df_filled[['enrollee_id', 'cdi_per']], on='enrollee_id', how='left')

# Show some combined insights:
# e.g., average cdi_per by education_level and relevent_experience
insight_edu = merged_df.groupby('education_level')['cdi_per'].mean().sort_values(ascending=False)
insight_relevant = merged_df.groupby('relevent_experience')['cdi_per'].mean().sort_values(ascending=False)

print("Average CDI% by Education Level:\n", insight_edu)
print("\nAverage CDI% by Relevant Experience:\n", insight_relevant)


Average CDI% by Education Level:
 education_level
Phd               91.139942
High School       78.861197
Masters           77.734853
Primary School    75.898204
Graduate          73.223451
Name: cdi_per, dtype: float64

Average CDI% by Relevant Experience:
 relevent_experience
Has relevent experience    76.447106
No relevent experience     72.221177
Name: cdi_per, dtype: float64
